In [106]:
%run setup_path.py
%load_ext autoreload
%autoreload 2

from gorbslam.common.utils import to_tum
from gorbslam.common.plotting_utils import create_ape_fig


from os import path

from gorbslam.orbslam_processor import ModelType, ORBSLAMProcessor

# TRAJECTORY_NAME = 'vabadussild'
TRAJECTORY_NAME = 'vabadusepst'
# TRAJECTORY_NAME = 'oldtown'

model_type = ModelType.NN
# model_type = ModelType.GBR
# model_type = ModelType.SVR
# model_type = ModelType.RFR
# model_type = ModelType.UMEYAMA

processor = ORBSLAMProcessor(path.join('~/orbslam-out', TRAJECTORY_NAME), model_type)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [97]:
# processor.initialize_model(overwrite=True)
processor.initialize_model()


In [98]:
processor.fit_trajectories()


30/30 [==============================] - 0s 716us/step


In [99]:
processor.save_trajectories()


In [105]:
ape_fig = processor.create_ape_plot_all().show()


In [101]:
# fig_2d = processor.create_2d_plot_utm()
# fig_2d.show()


In [102]:
# fig_slam = processor.create_2d_plot_slam()
# fig_slam.show()


In [103]:
# fig_map = processor.create_map_plot()
# fig_map.show()
